In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from time import perf_counter

In [2]:
filename = './datasets/facebook_combined.txt'

In [3]:
t0 = perf_counter()
testgraph = nx.read_edgelist(filename, comments='#', delimiter=None, create_using=None, nodetype=None, data=True, edgetype=None, encoding='utf-8')
t_load = perf_counter() - t0

In [4]:
full_adj = nx.adjacency_matrix(testgraph)
print(full_adj.shape)

(4039, 4039)


/tmp/ipykernel_706344/1967707738.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  full_adj = nx.adjacency_matrix(testgraph)


In [5]:
full_adj_dense = full_adj.todense()
print(full_adj_dense.shape)

(4039, 4039)


In [6]:
part_1_adj = full_adj_dense[:, 0:2000]
part_2_adj = full_adj_dense[:, 2001:full_adj_dense.shape[1]]
print(part_1_adj.shape[0], part_1_adj.shape[1])
print(part_2_adj.shape)

4039 2000
(4039, 2038)


In [7]:
part_1_matrix = np.zeros((full_adj.shape))
part_2_matrix = np.zeros((full_adj.shape))
print(part_1_matrix.shape)
print(part_2_matrix.shape)

(4039, 4039)
(4039, 4039)


In [ ]:
for i in range (0, part_1_adj.shape[0]):
    for j in range(0, part_1_adj.shape[1]):
        part_1_matrix[i,j] = part_1_adj[i,j]
        
for i in range (0, part_2_adj.shape[0]):
    for j in range(0, part_2_adj.shape[1]):
        part_2_matrix[i, j] = part_2_adj[i,j]

In [ ]:
part_1_graph = nx.from_numpy_matrix(part_1_matrix)
part_2_graph = nx.from_numpy_matrix(part_2_matrix)

In [ ]:
count = 0
triangle = []
for edge_list in testgraph.edges:
#     if int(edge_list[1]) < 2000:
    srcSet = part_1_graph.adj[int(edge_list[0])]
    dstSet = part_1_graph.adj[int(edge_list[1])]
    for vertexID in dstSet:
        if (vertexID in srcSet): # and (vertexID > int(edge_list[1])):
            count += 1
            triangle.append((edge_list[0], edge_list[1], vertexID))
print("part_1 triangles: ", count) 

In [ ]:
count_2 = 0
triangle_2 = []
for edge_list in testgraph.edges:
#     if int(edge_list[1]) >= 2000:
    srcSet = part_1_graph.adj[int(edge_list[0])]
    dstSet = part_2_graph.adj[int(edge_list[1])]
    for vertexID in dstSet:
        if (vertexID in srcSet): # and (vertexID > int(edge_list[1])):
            count_2 += 1
            triangle_2.append((edge_list[0], edge_list[1], vertexID))
print("part_2 triangles: ", count_2)
print("Triangles in total:", count + count_1 + count_2)